## The Simple Example for Document-oriented Conversational Bot

In [13]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, format_document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from chat_model_client import get_model
from pprint import pprint

1. 加载页面内容

In [17]:
loader = WebBaseLoader("https://app.xinhuanet.com/news/article.html?articleId=cf58e389142893239ff9e986e7a3e06a")
docs = loader.load()
pprint(docs)

[Document(metadata={'source': 'https://app.xinhuanet.com/news/article.html?articleId=cf58e389142893239ff9e986e7a3e06a', 'title': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力 - 新华网客户端', 'description': '中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力----新华网北京1月7日电（侯大伟 李旭）一年消费总规模超133亿元、单个赛事报名人数达450744人……日前刚刚结束的中国马拉松博览会上，多组数据让行业备受鼓舞；近期相关赛事的报名数据更是将以马拉松为代表的中国路跑运动送上热点。45万+的报名人数虽属罕见，但整个2024年，动辄20万+的报名数字在国内头部马拉松赛事中已屡见不鲜。 2014年，《国务院关于加快发展体育产业促进体育消费的若干意见》颁布，自此，以马拉松为代表的中国路跑运动迎来黄金十年，不断倍增的报名人数成为“跑马热”最直观的印证。“周末两眼一睁就是马拉松，朋友圈被越来越多去全国各地参赛的跑友‘刷屏’了！”来自北京的跑者王鹏表示。 中国马拉松官网数据显示，2024年，全国共举办路跑赛事671场，参赛人次约656万。赛事分布范围涵盖了全国31个省级行政区，261个市，537个区县。仅2024年11月，国内就有114场赛事鸣枪举办，其中1', 'language': 'zh'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力 - 新华网客户端\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n新华网客户端\n\n\n下载新华网客户端\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力\n\n\n新华网客户端\n\n\n2025-01-07 13:29:11\n\n\n\n180.8万\n\n\n\n\n新华网北京1月7日电（侯大伟 李旭）一年消费总规模超133亿元、单个赛事报名人数达450744人……日前刚刚结束的中国马拉松博览会上，多组数据让行业备受鼓舞；近期相关赛事的报名数据更是将以马拉松为代表的中国路跑运动送上热点。

2. 把文本按500个字符分割成片段

In [ ]:
spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
chunks = spliter.split_documents(docs)
pprint(len(chunks))

18


3. 构建Chroma向量数据存储，并生成对应的Retriever

In [19]:
vs = Chroma.from_documents(chunks, OllamaEmbeddings(model = "llama2-chinese"))
retriever = vs.as_retriever()

4. 合并文本段落

In [20]:
document_prompt = PromptTemplate.from_template("{page_content}")

def _combine_documents(docs, document_prompt = document_prompt, document_separator="\n\n"):
    doc_string = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_string)

5. 准备Model I/O三元组，并构建RAG链

In [21]:
prompt_template = """
    基于以下内容回答问题：
    {context}

    Question: {question}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
llm = get_model("llama")
chain = (
    {
        "context": retriever | _combine_documents,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

6. 询问

In [22]:

response = chain.invoke("马拉松的经济价值")
pprint(response)

('基于以上内容回答问题：马拉松的经济价值。\n'
 '\n'
 '根据2024年1月至7月，体育旅游的搜索热度增长87%，户外运动参与人数同比增长19%。马拉松、骑行等户外赛事带来的“流量”，亦是城市旅游的“留量”，正成为跑步赛事旅行的新特点。国家体育总局体育经济司发布的《中国户外运动产业发展报告(2023-2024)》显示，一场大型城市马拉松赛事带动的经济效益可达6-7亿元。\n'
 '\n'
 '此外，根据2025年3月举办的武汉马拉松，将于2025年3月举办的武汉马拉松，更是再一次开创报名人数新历史，一举将中国马拉松赛事报名人数拉高到45万+……“这两年很多大型马拉松赛事报名人数屡创新高，一些城市不断扩大赛事规模。”国家体育总局体育科学研究所中国体育经济研究中心日前发布的《特征与趋势洞察：中国跑者的运动参与和消费》进一步印证了这一观点：在2023年中国路跑赛事运落情况抽样调查，一线城市举办的大型路跑赛事平均收入为2910.24万元（含报名、赞助、财政补助等）；其他类型城市举办的路跑赛事平均收入为844.88万元。马拉松赛事自身的经济效益已毋庸置疑。\n'
 '\n'
 '因此，可以确定出马拉松赛事的经济价值在2023年达到了6-7亿元，并且这个数字将会不断地上升。同时，由于马拉松赛事自身的经济效益已毋庸置疑，因此可以确定出马拉松赛事的经济价值在2023年达到了6-7亿元。\n')


7. 询问

In [23]:
response = chain.invoke("上海马拉松的报名人数是多少？")
pprint(response)

('\n'
 '根据文章中提到的，2025年厦门马拉松的报名人数为3.5万人，而武汉马拉松则一举创造了45万+的报名人数。因此，上海马拉松的报名人数可能是多少？\n'
 '\n'
 '根据文章中提到的，中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力，这些都表明了中国马拉松产业的不断发展和扩大。因此，上海马拉松也可能在未来几年内成为一个热门的赛事，并有更多的人报名参加。\n')


8. 链路检查

In [24]:
pprint(chain.get_graph().print_ascii())

             +---------------------------------+          
             | Parallel<context,question>Input |          
             +---------------------------------+          
                    ***                ***                
                 ***                      ***             
               **                            ***          
+----------------------+                        **        
| VectorStoreRetriever |                         *        
+----------------------+                         *        
            *                                    *        
            *                                    *        
            *                                    *        
 +--------------------+                   +-------------+ 
 | _combine_documents |                   | Passthrough | 
 +--------------------+                   +-------------+ 
                    ***                ***                
                       ***          ***                 